# Modelo de una compuerta

## Cambio de estado

La probabilidad de que una compuerta esté abierta, como función del tiempo, es una combinación lineal de las probabilidades de que la compuerta se halla cerrado o abierto y las reglas observadas de su comportamiento se describen con una ecuación diferencial.

Este comportamiento se ve afectado por el voltaje de polarización de la membrana, pero para esta sección consideraremos al voltaje como un parámetro fijo, de modo que nos podamos enfocar en la ecuación como función del tiempo.  Tomemos como ejemplo la ecuación para la compuerta de potasio (K).

El cambio en la probabilidad de que una compuerta esté abierta se puede decir que es proporcional a:

\begin{align*}
 \Delta \text{Compuertas abiertas} = \text{Puertas que se abren} - \text{Puertas que se cierran}
\end{align*}

Formalizando, es la suma de la probabilidad de que la compuerta esté cerrada por la probabilidad de que se abra, menos la probabilidad de que haya estado abierta y se cierre. Esto se expresa con una ecuación diferencial:

\begin{equation}
 \frac{1}{\gamma(T)} \frac{dn}{dt} = \alpha_n(V) (1 - n) - \beta_n(V)n \label{eq:huxley_sodio_puerta}
\end{equation}

donde $\gamma(T)$ es una constante de proporcionalidad temporal que depende de la temperatura $T$, a $6.3°$C vale $1$ [Cessac2009].

### Estado de equilibrio

La ecuación anterior es una ecuación diferencial de primer grado, si bien la ecuación describe la variación de $n$ en el tiempo, existe un punto especial en el cual $n$ ya no cambia.  A éste se le llama un **estado de equilibrio**, matemáticamente corresponde a que su derivada sea cero.

\begin{align}
  \frac{dn}{dt} &= 0
\end{align}

Si sustituimos esta condición en la ecuación anterior obtenemos el valor que toma $n$ al alcanzar el equilibrio, al cual llamaremos $n_{\infty}$; esto como función del voltaje de polarización actual $V$:
\begin{align}
 \alpha_n(V) (1 - n_{\infty}) - \beta_n(V)n_{\infty} &= 0 \\
 n_{\infty}(V) &= \frac{\alpha_n(V)}{\alpha_n(V) + \beta_n(V)}
\end{align}


### Constante temporal

Se define ahora a la constante temporal:
\begin{align*}
 \tau_n(V) &= \frac{1}{\alpha_n(V) + \beta_n(V)}
\end{align*}

### Ecuación de transición

Es costumbre reescribir la ecuación diferencial original en términos de $n_{\infty}$ y $\tau_n$:

\begin{align}
 \frac{1}{\gamma(T)} \frac{dn}{dt} &= \frac{n^{\infty}(V) - n}{\tau_n(V)} \label{eq:huxley_sodio_puerta}
\end{align}